# **Interactive Campaign Optimization Simulator**
This notebook provides a game that lets you interactively optimize a campaign manually and compares your results to the state of the art Multi-Armed Bandit algorithms $\epsilon$-greedy, upper confidence bounds (UCB), or Thompson Sampling.

The intricacy in optimizing campaigns lies in the fact that the teaser $ctr$'s (and also other KPIs) are not known with full certainty. If the $ctr$'s would be exactly known, the optimization problem would be trivial (play the best teaser $100\%$ of the time).

Instead of a pure optimization problem, the task is on the one hand to maximize the number of clicks, but on the other to simultaneously try out different configurations in a clever way to acquire new data (clicks & views) that are most informative to the true optimum.

This is often referred to as the *Exploration-Exploitation Dilemma*: we need to *explore* and play different teasers to see which one of them works best. But at the same time, we need to *exploit* our current knowlege and play the configuration we assume to be best as often as possible.


### **Before you execute the first cell**
Before you execute the first cell, go to your Google Drive and under "Shared with me" ("Für mich freigegeben"), right click on the file "simulator.py" that was shared with you and click on "Add shortcut to Drive" ("Drive-Verknüpfung hinzufügen"). Choose "My Drive" ("Meine Ablage"). Now you are ready to go!

To execute a cell, you can either click the "Play" button on the left or click on the cell and press "Shift + Enter". Some cells depend on each other, so please execute the cells from top to bottom!


**Execute the first cell to load all the necessary Python modules.**

In [2]:
from google.colab import drive
drive.mount('drive/')
!cp "drive/My Drive/simulator.py" .
import sys
sys.path.append('/content/')
from simulator import HumanAgent, EpsilonGreedyAgent, ThompsonSamplingAgent, UCBAgent, NoOptimizationAgent, OptimizationGame
import scipy.stats as spst
import numpy as np
from numpy.random import default_rng
rng = default_rng()
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

ModuleNotFoundError: No module named 'google'

### **Setting up a campaign environment**
You first need to specify how many teasers your campaign should have by specifying `n_teasers = X`. You can also manually specify the true teaser ctr's by setting a list like `true_ctrs = [.0005, .001, .0012, .003, .002, .004]`. If you do not specify anything, some random ctr's are sampled with a mean around ~0.2%.

**Important note:** The true ctr's are only necessary to generate mock click data. Neither you nor any of the optimization algorithms is allowed to use this information for optimization!

In [ ]:
n_teasers = 6
dist_alpha = 2.0
dist_beta = 1000.0
true_ctrs = rng.beta(a=dist_alpha, b=dist_beta, size=n_teasers)
# true_ctrs = [.0005, .001, .0012, .003, .002, .004]

## **Setting up the agents**
<img src="https://upload.wikimedia.org/wikipedia/commons/1/1b/Reinforcement_learning_diagram.svg?export=view">

In reinforcement learning, an agent is a human or machine that observes the environment (i.e., that tracks the teaser clicks and views) and based on the data observed, performes actions (turns teasers on/off) to increase what is called *reward* (= the number of clicks).

For this notebook, there are different kinds of agents implemented based on the strategy they play to increase the number of clicks. Parameters that all agents have in common are

*   `n_teasers=X` : The number of teasers an agent should optimize.

*   `n_views_update_data=X` : This specifies the number of views that happen until the agent gets aware of fresh data. This corresponds to the daily import of tracking data from the ad server. The default is 10 000, i.e., every 10 000 views a daily import happens and the agent updates its knowledge about the teasers.
*   `n_views_update_configuration=X` : This is the amout of views after which the agent can modify the configuration. If we assume 10 000 views per day, for an automated agent it would be reasonable to specify `n_views_update_configuration=500` to modify the configuration $\approx 20 \times$ a day. For a human agent, however, it would be more reasonable to set `n_views_update_configuration=70000`, which would correspond to an update of the configuration only once a week.

### **The NoOptimizationAgent**
The `NoOptimizationAgent` does nothing. It keeps all teasers active until the end of the game and only tracks the clicks and views of every teaser. Let's set one up for comparison:

In [ ]:
no_optimization_agent = NoOptimizationAgent(n_teasers=n_teasers)

### **The HumanAgent**
The `HumanAgent` pauses the game after every `n_views_update_configuration` views, prints the currently known teaser clicks, views and the empirical click rate on the screen (from all the daily imports that have happened so far) and asks the user to activate or deactivate the specific teasers. Let's set it up here:

In [ ]:
human_agent = HumanAgent(n_views_update_data=10000, n_views_update_configuration=70000, n_teasers=n_teasers)

### **The EpsilonGreedyAgent**
The strategy of the `EpsilonGreedyAgent` is to play the teaser with highest empirical $ctr$ (i.e., recorded $\frac{N_{clicks}}{N_{views}}$) with a probability $p = 1 - \epsilon$, whereas with a probability $p = \epsilon$, one of the other teasers is played. This simple algorithm works very well in practice, but the $\epsilon$ needs to be fine-tuned manually for every problem. We set up a $5\%$-greedy algorithm for our game:

In [ ]:
e_greedy_agent = EpsilonGreedyAgent(epsilon=.05, n_teasers=n_teasers, n_views_update_data=10000, 
                                    n_views_update_configuration=500)

### **The ThompsonSamplingAgent**
The `ThompsonSamplingAgent` tracks the teaser clicks and views and uses Bayes' law to compute probability distributions over the teaser ctr's that reflect the current state of knowledge ([for details, check the Insights documentation here](https://content-garden.gitlab.io/analytics-server/theory.html#thompson-sampling)). It then draws random realizations of the $ctr$'s from these distributions and performs the optimal action given these realizations, which is to only activate the teaser with highest sampled $ctr$. Additional parameters that can be set are

*   `prior_alpha=X, prior_beta=Y` : These parameters tell the agent what is known about the teaser $ctr$'s *before* any clicks and views have been observed, e.g., we may already know that teaser $ctr$'s are around $\approx 0.2\%$, ranging from $0.05\%$ to $0.5\%$, which would correspond to specific values for `prior_alpha, prior_beta`. It is reasonable to use the same values as were used to draw the `true_ctrs`, i.e., `prior_alpha=dist_alpha, prior_beta=dist_beta`.

Let's initialize a `ThompsonSamplingAgent`:

In [ ]:
ts_agent = ThompsonSamplingAgent(n_views_update_data=10000, n_views_update_configuration=500, n_teasers=n_teasers,
                                 prior_alpha=dist_alpha, prior_beta=dist_beta)

We also set up a `ThompsonSamplingAgent` with real-time tracking and immediate configuration updates:

In [ ]:
perfect_ts_agent = ThompsonSamplingAgent(n_views_update_data=1, n_views_update_configuration=1, n_teasers=n_teasers,
                                 prior_alpha=dist_alpha, prior_beta=dist_beta)

Moreover, a-priori information on the teaser performance can be passed to the Thompson Sampling Agent. Assume we have a machine learning model that provides predictions on the teaser $ctr$. To mock this, we assume that the predictive mean is equal to the true $ctr$ whereas the predictive variance roughly stays the same:

In [ ]:
prior_alpha = dist_beta * true_ctrs/(1.0 - true_ctrs)
a_priori_ts_agent = ThompsonSamplingAgent(n_views_update_data=1, n_views_update_configuration=1, n_teasers=n_teasers,
                                 prior_alpha=prior_alpha, prior_beta=dist_beta)

### **The UCBAgent**
The `UCBAgent` uses the same probability distributions over the ctr's as are used by the `ThompsonSamplingAgent` but instead of sampling from these distributions, it uses them to compute **U**pper **C**onfidence **B**ounds, typically the $95\%$ quantile. It then plays the teaser with highest upper confidence bound, therefore automatically balancing exploration and exploitation. It has the same parameters as the `ThompsonSamplingAgent` plus `ucb_quantile` which specifies the quantile used as an upper confidence bound.

In [ ]:
ucb_agent = UCBAgent(n_views_update_data=10000, n_views_update_configuration=500, n_teasers=n_teasers,
                                 prior_alpha=dist_alpha, prior_beta=dist_beta, ucb_quantile=0.95)

## **Setting up the OptimizationGame**
Finally, we need to set up an `OptimizationGame` which has the following parameters:

*   `agents=[...]` : A list of all the agents that have been set up and should participate in the game.
*   `true_ctrs=X` : The true ctr's that have been specified above. **These are not known to the agents!** They are only necessary to generate mock clicks and views.
*   `n_views_total=X` : The total amount of teaser views after which the game should end.

In [ ]:
game = OptimizationGame([human_agent, no_optimization_agent, e_greedy_agent, ucb_agent, ts_agent, perfect_ts_agent, a_priori_ts_agent],
                        true_ctrs=true_ctrs, n_views_total=250000)

## **Before you start**
It is important to note that randomness plays a key role in this game. As all agents may activate/deactivate different teasers in every step, they necessarily need to run on different random sequences (i.e., different clicks and views).

**It is possible that you will win a game from time to time!**

It would however be surprising if you win 6 out of 10 or even 51 out of 100 games.

## **Start the game**
Execute the next cell to run the game. Before the first view, you will be asked to activate the teasers you want to play. Is it reasonable to activate all the teasers in the beginning?

Then, after every `n_views_update_configuration` views you have set up for the `human_agent`, the currently known stats are shown and you can modify the configuration. Also, a short summary of the accumulated clicks for all agents is printed.

In [ ]:
game.start()
print('true ctr`s: ', true_ctrs)
ts_agent.update_data()
perfect_ts_agent.update_data()
no_optimization_agent.update_data()
human_agent.update_data()
ts_tot_clicks = ts_agent.n_clicks.sum()
perfect_ts_tot_clicks = perfect_ts_agent.n_clicks.sum()
no_opt_tot_clicks = no_optimization_agent.n_clicks.sum()
human_tot_clicks = human_agent.n_clicks.sum()
print('Improvement  no  optimization   -->     TS: ' + '%2.2f' % (100 * (ts_tot_clicks/no_opt_tot_clicks - 1.0)) + '%')
print('Improvement      human          -->     TS: ' + '%2.2f' % (100 * (ts_tot_clicks/human_tot_clicks - 1.0)) + '%')
print('Improvement no optimization --> perfect TS: ' + '%2.2f' % (100 * (perfect_ts_tot_clicks/no_opt_tot_clicks - 1.0)) + '%')
print('Improvement    human    -->     perfect TS: ' + '%2.2f' % (100 * (perfect_ts_tot_clicks/human_tot_clicks - 1.0)) + '%')

# **Explanation of Algorithms**
## **$\epsilon$-greedy**
The idea of the $\epsilon$-greedy algorithm is simple: With probability $1 - \epsilon$, play the teaser which has currently the highest empirical $ctr$ (exploitation step). With probability $\epsilon$, randomly choose one of the other teasers (exploration). The value of $\epsilon$ needs to be set by the user. High $\epsilon$ values drive exploration, low $\epsilon$ values drive exploitation. Reasonable values for $\epsilon$ are $1\% \lesssim \epsilon \lesssim 10\%$. Let's play an optimization game with an $\epsilon$-greedy agent only and look more closely what it's doing by setting the option `verbose=True`. The empirical $ctr$'s are visualized in chart of blue bars. The teaser that will be activated until the next configuration update is highlighted in orange color. We use $\epsilon = 20\%$ to observe an exploration step more often:

In [ ]:
e_greedy_agent = EpsilonGreedyAgent(epsilon=.2, n_teasers=n_teasers, n_views_update_data=10000,
                                    n_views_update_configuration=3000, verbose=True)
game = OptimizationGame([e_greedy_agent], true_ctrs=true_ctrs, n_views_total=1000000)
game.start()

$\epsilon$-greedy is a very simple algorithm that shows surprisingly good results. However, in the limit of infinite views, the best teaser is not played with a probability of $\epsilon$ even though it is known with certainty. A schedule to decrease $\epsilon$ over time can be applied, but a badly chosen schedule can completely mess up the optimization.

## **Thompson Sampling**
Thompson Sampling uses Bayes' law to compute distributions over the $ctr$'s given a certain amount of clicks and views ([for details, check the Insights documentation here](https://content-garden.gitlab.io/analytics-server/theory.html#thompson-sampling)). It then plays a teaser with the same probability as the probability of this teaser to be the best one ("probability matching decision strategy"). It achieves this by drawing random numbers from the inferred probability distributions over the $ctr$'s and then playing the teaser corresponding to the highest random number that was drawn.

Assume that, after recording some clicks and views for, say, 3 different teasers, we know that the first one has $10\%$, the second one $20\%$ and the third one $70\%$ probability to be the best teaser. Then Thompson Sampling will play the first one $10\%$, the second one $20\%$ and the third one $70\%$ of the time.

After some more views, our knowledge of the true teaser $ctr$'s increases and it may be that according to our knowledge, the first teaser is with $2\%$, the second with $5\%$ and the third with $93\%$ the best teaser. Then the first one is played $2\%$, the second $5\%$ and the third one $93\%$ of the time.

It may as well be that actually the second teaser is better than the third and there was only bad luck for the second one in the first couple of views. Then our knowlege about the best teaser may refine to, e.g., $4\%$ for the first, $60\%$ for the second and $36\%$ for the third and the Thompson Sampling agent will play the teasers accordingly.

In the limit of infinite views, Thompson Sampling will always find the best teaser and play it almost $100\%$ of the time in future steps.

A nice feature is that this algorithm does not require any fine tuning of parameters as is necessary in, e.g., the $\epsilon$-greedy method. 

Let's set up a Thompson Sampling agent and set `verbose=True` to see some visualizations. Observe that whenever data is imported ("daily import"), the probability distributions over the teaser $ctr$'s change. Whenever the configuration is updated, random numbers from these distributions are drawn and visualized as dots on the $x$-axis of the same color as the probability distribution of the corresponding teaser $ctr$. Only the teaser with the largest drawn random number is activated, indicated by an encircled dot and a bigger line width. 

In [ ]:
ts_agent = ThompsonSamplingAgent(n_views_update_data=10000, n_views_update_configuration=3000, n_teasers=n_teasers,
                                 prior_alpha=dist_alpha, prior_beta=dist_beta, verbose=True)
game = OptimizationGame([ts_agent], true_ctrs=true_ctrs, n_views_total=1000000)
game.start()

## **Upper Confidence Bounds**
The UCB algorithm is pretty similar to Thompson Sampling, but instead of drawing random numbers from the inferred probability distributions over the teaser $ctr$'s, upper confidence bounds are computed and the teaser with largest value is selected. A common upper confidence bound is the $95\%$ quantile (i.e., the $ctr$ value where to $95\%$ probability, the true $ctr$ is smaller, and to $5\%$ it's bigger).

We specify a UCB agent, draw the probability distributions over the $ctr$'s at every configuration update, and indicate the upper confidence bounds ($95\%$ quantiles) with dots on the $x$-axis.

In [ ]:
ucb_agent = UCBAgent(n_views_update_data=10000, n_views_update_configuration=3000, n_teasers=n_teasers,
                                 prior_alpha=dist_alpha, prior_beta=dist_beta, ucb_quantile=0.95, verbose=True)
game = OptimizationGame([ucb_agent], true_ctrs=true_ctrs, n_views_total=1000000)
game.start()

The UCB algorithm shows similar or even better performance than Thompson Sampling. The issue in our context is that it will always play the same teaser until fresh data is imported, i.e., only a single active teaser for a whole day.

# **Summary**
The three most common Multi-Armed Bandit algorithms ($\epsilon$-greedy, UCB, and Thompson Sampling) were presented. The Bayesian methods UCB and Thompson Sampling generally show the best performance. However, applying UCB would in our case mean that only a single teaser is active on a campaign/placement pair for a whole day since configurations do not change until there is fresh data.

Therefore, in our context, Thompson Sampling seems to be most promising.

To better align Thompson Sampling with the restriction of only limited configuration updates, strategies with more than a single active teaser can be developed.

# **Appendix: Probability Distribution Plotter**
The next cell provides a plotting tool to visualize the probability distribution over the teaser $ctr$ given a certain amount of clicks and views. Specify `n_clicks` and `n_views` and run the cell to see the distribution!

In [ ]:
n_clicks = 1
n_views = 1000

prior_alpha = 1.1
prior_beta = 300.0
posterior_alpha = prior_alpha + n_clicks
posterior_beta = prior_beta + n_views - n_clicks
ctr = np.linspace(.0, .015, 501)
p_ctr = spst.beta.pdf(ctr, posterior_alpha, posterior_beta)

fig = figure(background_fill_color=(255, 255, 255), plot_width=1000, plot_height=300, 
             tools="hover,pan,wheel_zoom,box_zoom,reset,crosshair,save", toolbar_location='right', 
             title='Probability Distribution over Teaser CTR')
fig.line(ctr, p_ctr, line_width=3)
fig.y_range.start = 0.0
fig.xaxis.major_label_text_font_size = "16px"
fig.yaxis.major_label_text_font_size = "16px"
fig.xaxis.axis_label_text_font_size = "16px"
fig.yaxis.axis_label_text_font_size = "16px"
fig.yaxis.axis_label = 'p(ctr)'
fig.xaxis.axis_label = 'ctr'
show(fig)